In [1]:
import pandas as pd

In [144]:
df = pd.read_csv('telegram_messages.csv')

df_reversed = df[::-1].reset_index(drop=True)

print(df_reversed.head(10))


        sender_name                                            message  \
0            масим                     я вас категорически приветствую   
1            масим                                     пойдешь гулять?   
2            масим   сегодня пятница оказывается, а вчера я думал, ...   
3            масим                                                 NaN   
4            масим                                                 NaN   
5            масим                                                   (   
6  Евгений Потурило                                           о привет   
7            масим                                            вооот он   
8            масим                                               хехех   
9            масим                                        месяц спустя   

                        date  
0  2021-07-23 15:40:55+00:00  
1  2021-07-23 15:41:03+00:00  
2  2021-07-23 15:42:03+00:00  
3  2021-07-23 17:56:30+00:00  
4  2021-07-23 18:00:43+00:00  

удаляем ненужный столбец

In [145]:
df = df_reversed
df = df[df['sender_name'].str.startswith("Leo") == False]
df = df[df['sender_name'].str.startswith("Плюс") == False]

In [146]:
df = df[df['sender_name'].str.startswith("СРОЧНО") == False]
df = df[df['sender_name'].str.startswith("ml_mipt_basic_faq_bot") == False]
df = df[df['sender_name'].str.startswith("TinkoffMath") == False]
df = df[df['sender_name'].str.startswith("sender_name") == False]
df = df[df['message'].str.startswith("sender_name") == False]
df = df[df['message'].isnull() == False]

In [147]:
len(df)

50813

In [97]:
df.head(100)

,sender_name,message,date
0,масим,я вас категорически приветствую,2021-07-23 15:40:55+00:00
1,масим,пойдешь гулять?,2021-07-23 15:41:03+00:00
2,масим,"сегодня пятница оказывается, а вчера я думал, ...",2021-07-23 15:42:03+00:00
5,масим,(,2021-07-23 18:00:51+00:00
6,Евгений Потурило,о привет,2021-08-18 18:38:53+00:00
...,...,...,...
127,Камал,Здравствуйте! Если хотите в пятницу встретимся.,2022-02-01 17:55:32+00:00
128,Евгений Потурило,Хорошо,2022-02-01 18:09:52+00:00
129,Евгений Потурило,"Здравствуйте, извините что с таким опозданием ...",2022-05-20 13:05:33+00:00
130,Камал,Спасибо,2022-05-20 13:23:51+00:00


In [151]:
df['date'] = pd.to_datetime(df['date'], format='ISO8601')

def combine_messages(group):
    combined_message = group.iloc[0]['message']
    last_date = group.iloc[0]['date']
    
    for _, row in group.iloc[1:].iterrows():
        if abs(row['date'] - last_date) <= pd.Timedelta(hours=1):
            combined_message += ' \n ' + row['message']
        else:
            break
        last_date = row['date']
    
    return pd.Series({
        'message': combined_message,
        'date': group.iloc[0]['date']
    })

result = df.groupby((df['sender_name'] != df['sender_name'].shift()).cumsum()).apply(combine_messages)

result['sender_name'] = df.groupby((df['sender_name'] != df['sender_name'].shift()).cumsum())['sender_name'].first()

result = result.reset_index(drop=True)

In [152]:
result.head()

,message,date,sender_name
0,я вас категорически приветствую \n пойдешь гул...,2021-07-23 15:40:55+00:00,масим
1,о привет,2021-08-18 18:38:53+00:00,Евгений Потурило
2,вооот он \n хехех \n месяц спустя,2021-08-18 18:45:39+00:00,масим
3,да блин не разобрался как зайти \n ты чо \n ка...,2021-08-18 18:46:05+00:00,Евгений Потурило
4,ага \n нормально мне \n а ты там как? \n в дер...,2021-08-18 18:46:21+00:00,масим


In [153]:
time_diff = result['date'].diff()

result['dialogue_index'] = (time_diff > pd.Timedelta(hours=16)).cumsum()

In [154]:
result.head(20)

,message,date,sender_name,dialogue_index
0,я вас категорически приветствую \n пойдешь гул...,2021-07-23 15:40:55+00:00,масим,0
1,о привет,2021-08-18 18:38:53+00:00,Евгений Потурило,1
2,вооот он \n хехех \n месяц спустя,2021-08-18 18:45:39+00:00,масим,1
3,да блин не разобрался как зайти \n ты чо \n ка...,2021-08-18 18:46:05+00:00,Евгений Потурило,1
4,ага \n нормально мне \n а ты там как? \n в дер...,2021-08-18 18:46:21+00:00,масим,1
5,не \n тут \n чилю,2021-08-18 18:46:42+00:00,Евгений Потурило,1
6,атлишна \n видел твои сторки в инсте,2021-08-18 18:46:59+00:00,масим,1
7,в пятницу пойдем с кириллом свертокой и женей ...,2021-08-18 18:47:11+00:00,Евгений Потурило,1
8,"в пятницу... это ж через два дня, хз еще, позж...",2021-08-18 18:47:58+00:00,масим,1
9,ну смари сам \n да в бар \n посидеть,2021-08-18 18:48:09+00:00,Евгений Потурило,1


Удаляем диалоги в которых 1 неотвеченное сообщение

In [155]:
dialogue_counts = result['dialogue_index'].value_counts()
result = result[result['dialogue_index'].isin(dialogue_counts[dialogue_counts > 1].index)]

result = result.reset_index(drop=True)


In [156]:
result.head()

,message,date,sender_name,dialogue_index
0,о привет,2021-08-18 18:38:53+00:00,Евгений Потурило,1
1,вооот он \n хехех \n месяц спустя,2021-08-18 18:45:39+00:00,масим,1
2,да блин не разобрался как зайти \n ты чо \n ка...,2021-08-18 18:46:05+00:00,Евгений Потурило,1
3,ага \n нормально мне \n а ты там как? \n в дер...,2021-08-18 18:46:21+00:00,масим,1
4,не \n тут \n чилю,2021-08-18 18:46:42+00:00,Евгений Потурило,1


In [157]:
def word_count(text):
    return len(str(text).split())

result['message_word_count'] = result['message'].apply(word_count)

# Считаем общее количество слов для каждого dialogue_index
word_count_per_dialogue = result.groupby('dialogue_index')['message_word_count'].sum()

# Добавляем новую колонку с общим количеством слов для каждого dialogue_index
result['total_words_in_dialogue'] = result['dialogue_index'].map(word_count_per_dialogue)

# Сбрасываем индекс, если это необходимо
result = result.reset_index(drop=True)

In [190]:
x = result.iloc[3301]["context"]
x = x.replace("user", "person")
x

'model : Почему \n Аххааа\nperson : из не постсоветского культурного пространства \n ну \n бэкграунд культурный слишком разный \n здесь любоваться панельками это типа ваще необычно \n и если ты слушаешь цоя ты типа крутой \n это не распространено \n типа \n это считается \n люди которые хотят показаться не такими как все так делают \n это пример \n а дома это классика \n вот \n поэтому так себе \n А мне важно типа что бы прям глубокая такая связь была \n душевная \n понимание и всё такое\nmodel : Я думал поляки похожи на нас \n В этом плане\nperson : ну мы сейчас ваще с Настей вместе, но как уже сказал там кризис\nmodel : А она в Польше стоп\nperson : Ну в сравнении с вьетнамцами - да\nmodel : Тому рб\nperson : А в сравнении с нашими - нет \n Она в Минске\nmodel : Блин прикольно \n Мне сложно пока вериться в это, но мб \n Я там не был считай\nperson : Что поляки не похожи?\nmodel : Ага \n Ну все мои знакомые вроде норм уехали и уехали \n Много белорусов туда укатило \n Вроде никто не в

Супер длинных диалогов нет, поэтому можно использовать все диалоги в качестве контекста не урезая

In [180]:
def create_context(group):
    context = []
    for idx, row in group.iterrows():
        speaker = "model" if row['sender_name'] == "Евгений Потурило" else "user"
        context.append(f"{speaker} : {row['message']}")
    
    group['context'] = ['\n'.join(context[:i - 1]) for i in range(1, len(context) + 1)]
    return group

# Применяем функцию create_context к каждой группе dialogue_index
result = result.groupby('dialogue_index').apply(create_context)

# Сбрасываем индекс
result = result.reset_index(drop=True)

/var/folders/dt/y7wk4rzx3dsg1r6_m1tgxhwc0000gn/T/ipykernel_24205/4278071811.py:11: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  result = result.groupby('dialogue_index').apply(create_context)


In [195]:
result.to_csv("final_telegram.csv", index=False)

Удаляем sender_name != Евгений Потурило
Удаляем len(message) > 50

In [52]:
import pandas as pd

result = pd.read_csv('final_telegram.csv')

result = result[result["sender_name"] == "Евгений Потурило"]
result = result[result.message_word_count < 50]
result['context'] = result['context'].fillna('')


ImportError: cannot import name 'tokenizer' from 'datasets' (/Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/lib/python3.12/site-packages/datasets/__init__.py)

In [51]:
from typing import List
def generate_prompt(example: pd.DataFrame) -> List[str]:
    """Generates a standardized message to prompt the model"""
    instr = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
### Instruction:
You have dialugue between model and user. You should act like model. Input is previous context. """
    example["new_context"] = instr + example["context"] + "### Response:" + example["message"]
    return example["new_context"].tolist()

prompt_list = generate_prompt(result)

In [54]:
from datasets import Dataset

def prepare_dataset(conversations):
    dataset = []
    for conv in conversations:
        parts = conv.split('### Response:')
        if len(parts) == 2:
            context, response = parts
            context = context.split("Input is previous context.")[0] + "Input is previous context." + " ".join(context.split("Input is previous context.")[1].split(" ")[-100:])
            dataset.append({
                "text": f'<context>{context.strip()}<response>{response.strip()}'
            })

    return Dataset.from_list(dataset)

telegram_dataset = prepare_dataset(prompt_list)

In [55]:
telegram_dataset.save_to_disk('./train_dataset')

{'text': '<context>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nYou have dialugue between model and user. You should act like model. Input is previous context.<response>о привет'}

In [57]:
telegram_dataset[0]

{'text': '<context>Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.\n### Instruction:\nYou have dialugue between model and user. You should act like model. Input is previous context.<response>о привет'}

ТЕПЕРЬ ДЕЛАЕМ ДАТАСЕТ 200К

In [364]:
!pip uninstall numpy scikit-learn
!pip install numpy scikit-learn

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
275415.80s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Would remove:
    /Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/bin/f2py
    /Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/lib/python3.12/site-packages/numpy-1.26.4.dist-info/*
    /Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/lib/python3.12/site-packages/numpy/*
Proceed (Y/n)? ^C
ERROR: Operation cancelled by user


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
275503.87s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


In [3]:
import random
from datasets import load_dataset, Dataset
import numpy as np
from sklearn.model_selection import train_test_split

/Users/evgeny/Desktop/study/bot_telegram_request_message/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
dataset = load_dataset('Den4ikAI/russian_dialogues_2', split='train')

In [28]:
data = dataset["sample"]

In [29]:
def format_dialogue(data):
    dialogue = []
    for i in range(0, len(data) - 1, 2):
        user_str = f"user : {data[i]}"
        model_str = f"model : {data[i+1]}"
        dialogue.append(f"{user_str}\n{model_str}")
    return dialogue[0]

In [41]:
data = data[:200000]
print(len(data))
formatted_data = list(map(format_dialogue, data))

200000


In [39]:
mean = np.zeros(10000)
for i in range(10000):
    mean[i] = len(formatted_data[i].split(" "))

print(mean.argmax())
print(mean.mean())
print(formatted_data[6880])
print(len(formatted_data[6880].split(" ")))

6880
24.3523
user : Пока они не выберут того, кому будут доверять все - им придется ходить по кругу. Ну и армия у них, по ходу, бляди, раз так легко в своих стреляют. 
model : Они не так легко в своих стреляют, если бы они хотели, они бы на БТРах своих сравняли бы все с землей, но не стали этого делать
59


In [42]:
train_data, val_data = train_test_split(formatted_data, test_size=0.1, random_state=42)

In [43]:
train_dataset = Dataset.from_dict({"text": train_data})
val_dataset = Dataset.from_dict({"text": val_data})

In [47]:
train_dataset[0]

{'text': 'user : Что в этом такого? Это смотря каким чувством юмора обладает дающий оценку...\nmodel : Но ведь она это сказала не дающему оценку, а мужу своему'}

In [44]:

train_dataset.save_to_disk('./train_dataset')
val_dataset.save_to_disk('./val_dataset')


Saving the dataset (1/1 shards): 100%|██████████| 20000/20000 [00:00<00:00, 525085.47 examples/s]
